In [22]:
#Load our required library
import numpy as np
import pandas as pd
import glob  # to find all the csv files
import dask.dataframe as dd
from sklearn.ensemble import RandomForestClassifier
from dask_ml.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [2]:
header_path = "C:/Users/caoti/github/572CookCounty/Data/Header_interim2.txt"

# Load headers into a list
with open(header_path, 'r') as f:
    headers = f.readline().strip().split(',')
#Fix text code error when reading header file for SegmentCode column

# Define a dictionary with the data types for each column
dtype_dict = {
    'PIN': 'int64',
    'TaxpayerMailingState': 'str',
    'TaxpayerPropertyCity': 'str',
    'TaxCode': 'int',
    'HomeownerExempt': 'int',
    'SeniorExempt': 'int',
    'SeniorFreezeExempt': 'int',
    'BillType': 'int',
    'AdjustedAmountDue1': 'float',
    'TaxAmountDue1': 'float',
    'CostAmountDue1': 'float',
    'CofENumber': 'str',
    'PastTaxSaleStatus': 'str',
    'TaxRate': 'float',
    'CondemnationStatus': 'int',
    'MunicipalAcquisitionStatus': 'int',
    'AcquisitionStatus': 'int',
    'ExemptStatus': 'int',
    'BankruptStatus': 'int',
    'RefundStatus': 'int',
    'ReturningVetExemptionAmount': 'float',
    'DisabledPersonExemptionAmount': 'float',
    'DisabledVetExemptionAmount': 'float',
    'DisabledPersonVetExemptionAmount': 'float',
    'HomeownerExemptAmount': 'float',
    'SeniorFreezeExemptAmount': 'float',
    'LongtimeHomeownersExemptAmount': 'float',
    'TaxYear': 'float',
    'LastPaymentDate1': 'str',
    'LastPaymentDate2': 'str'
}

In [3]:
files = glob.glob('C:/Users/caoti/github/572CookCounty/Data/Cleaned_data/Property/20*.csv')
chunksize = 10 ** 5  # This is about 100,000 rows, 10^5 per files
#headers is a list of header
chunks = []

for file in files:
    for chunk in pd.read_csv(file, chunksize=chunksize, dtype=dtype_dict):
        chunks.append(chunk)
df = pd.concat(chunks, axis=0)
del chunks
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11191848 entries, 0 to 1866347
Data columns (total 31 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   PIN                               int64  
 1   TaxpayerMailingState              object 
 2   TaxpayerPropertyCity              object 
 3   TaxCode                           int32  
 4   HomeownerExempt                   int32  
 5   SeniorExempt                      int32  
 6   SeniorFreezeExempt                int32  
 7   BillType                          int32  
 8   AdjustedAmountDue1                float64
 9   TaxAmountDue1                     float64
 10  CostAmountDue1                    float64
 11  CofENumber                        object 
 12  PastTaxSaleStatus                 object 
 13  TaxRate                           float64
 14  CondemnationStatus                int32  
 15  MunicipalAcquisitionStatus        int32  
 16  AcquisitionStatus                 i

In [4]:
df.head(2)

,PIN,TaxpayerMailingState,TaxpayerPropertyCity,TaxCode,HomeownerExempt,SeniorExempt,SeniorFreezeExempt,BillType,AdjustedAmountDue1,TaxAmountDue1,...,DisabledPersonExemptionAmount,DisabledVetExemptionAmount,DisabledPersonVetExemptionAmount,HomeownerExemptAmount,SeniorFreezeExemptAmount,LongtimeHomeownersExemptAmount,TaxYear,BillYear,LastPaymentDate1,LastPaymentDate2
0,19084140790000,IL,CHICAGO,72001,1,0,0,1,133777.0,0.0,...,0.0,0.0,0.0,10000.0,0.0,0.0,2017.0,2017,03162018,08012018
1,19084140800000,IL,CHICAGO,72001,1,1,1,1,8952.0,0.0,...,0.0,0.0,0.0,10000.0,28515.0,0.0,2017.0,2017,02102018,00000000


In [5]:
#Number of unique code
TaxCode_num = df.loc[:,"TaxCode"].nunique()
#Number of unique property city
TaxpayerPropertyCity_num = df.loc[:,"TaxpayerPropertyCity"].nunique()

print("Number of unique tax code: {} and number of unique TaxPayerPropertyCity: {}".format(TaxCode_num,TaxpayerPropertyCity_num))

Number of unique tax code: 3652 and number of unique TaxPayerPropertyCity: 155


In [6]:
#Load in payment data
files = glob.glob('C:/Users/caoti/github/572CookCounty/Data/Cleaned_data/Payment/TY*.csv')
chunksize = 10 ** 5  # This is about 100,000 rows, 10^5 per files
#headers is a list of header
chunks = []

for file in files:
    for chunk in pd.read_csv(file, chunksize=chunksize):
        chunks.append(chunk)
df_payment = pd.concat(chunks, axis=0)
del chunks
df_payment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18709277 entries, 0 to 1614935
Data columns (total 17 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Payment       object 
 1   Volume        float64
 2   PIN           float64
 3   TaxType       float64
 4   TaxPayer      object 
 5   TaxYear       float64
 6   DatePaid      float64
 7   TaxPaid       float64
 8   InterestPaid  float64
 9   CostPaid      float64
 10  SerialNumber  float64
 11  SourceID      float64
 12  DateUpdated   object 
 13  RefundNumber  int64  
 14  TotalPaid     float64
 15  TaxYear4      float64
 16  WarrantYear   float64
dtypes: float64(13), int64(1), object(3)
memory usage: 2.5+ GB


In [7]:
df_payment.head(3)

,Payment,Volume,PIN,TaxType,TaxPayer,TaxYear,DatePaid,TaxPaid,InterestPaid,CostPaid,SerialNumber,SourceID,DateUpdated,RefundNumber,TotalPaid,TaxYear4,WarrantYear
0,P1,602.0,6021415.0,9.0,RAILROAD,17.0,180214.0,7282.52,0.0,0.0,2.151830e+10,501.0,2018-02-16 05:31:00,0,7282.52,2017.0,2017.0
1,P2,602.0,6021415.0,9.0,RAILROAD,17.0,180724.0,4191.36,0.0,0.0,7.251830e+10,501.0,2018-07-26 05:31:00,0,4191.36,2017.0,2017.0
2,P1,602.0,6021421.0,9.0,RAILROAD,17.0,180214.0,6970.00,0.0,0.0,2.151830e+10,501.0,2018-02-16 05:31:00,0,6970.00,2017.0,2017.0


In [8]:
#Read in payment due date file
instDates = pd.read_csv('C:/Users/caoti/github/572CookCounty/Data/Cleaned_data/Payment/InstDates.csv')
instDates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Year    46 non-null     int64 
 1   Inst1   46 non-null     object
 2   Inst2   45 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.2+ KB


In [9]:
#Reduce installment date to only the focus year 2017-2022 and renamed the column
instDates = instDates[instDates.loc[:,"Year"]>=2017]
instDates.rename(columns={'Year': 'TaxYear','Inst1': 'PmtDue1', 'Inst2': 'PmtDue2'}, inplace=True)
instDates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 40 to 45
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   TaxYear  6 non-null      int64 
 1   PmtDue1  6 non-null      object
 2   PmtDue2  5 non-null      object
dtypes: int64(1), object(2)
memory usage: 192.0+ bytes


In [10]:
#Join both property and payment due date
df_join = df.merge(instDates, on="TaxYear", how='left')
df_join.head(2)

,PIN,TaxpayerMailingState,TaxpayerPropertyCity,TaxCode,HomeownerExempt,SeniorExempt,SeniorFreezeExempt,BillType,AdjustedAmountDue1,TaxAmountDue1,...,DisabledPersonVetExemptionAmount,HomeownerExemptAmount,SeniorFreezeExemptAmount,LongtimeHomeownersExemptAmount,TaxYear,BillYear,LastPaymentDate1,LastPaymentDate2,PmtDue1,PmtDue2
0,19084140790000,IL,CHICAGO,72001,1,0,0,1,133777.0,0.0,...,0.0,10000.0,0.0,0.0,2017.0,2017,03162018,08012018,2018-03-01,2018-08-01
1,19084140800000,IL,CHICAGO,72001,1,1,1,1,8952.0,0.0,...,0.0,10000.0,28515.0,0.0,2017.0,2017,02102018,00000000,2018-03-01,2018-08-01


In [11]:
#Notice there are some instances with no LastPaymentDate1. Note tax exempt
df_concern1= df_join[(df_join.loc[:,"LastPaymentDate1"]=="00000000") & (df_join.loc[:,"AdjustedAmountDue1"]>0)]
df_concern1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 358276 entries, 1007 to 11191709
Data columns (total 33 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   PIN                               358276 non-null  int64  
 1   TaxpayerMailingState              358276 non-null  object 
 2   TaxpayerPropertyCity              358276 non-null  object 
 3   TaxCode                           358276 non-null  int32  
 4   HomeownerExempt                   358276 non-null  int32  
 5   SeniorExempt                      358276 non-null  int32  
 6   SeniorFreezeExempt                358276 non-null  int32  
 7   BillType                          358276 non-null  int32  
 8   AdjustedAmountDue1                358276 non-null  float64
 9   TaxAmountDue1                     358276 non-null  float64
 10  CostAmountDue1                    358276 non-null  float64
 11  CofENumber                        358276 non-nu

In [12]:
df_concern1.loc[df_concern1.loc[:, "PIN"] == 19103250470000, :]

,PIN,TaxpayerMailingState,TaxpayerPropertyCity,TaxCode,HomeownerExempt,SeniorExempt,SeniorFreezeExempt,BillType,AdjustedAmountDue1,TaxAmountDue1,...,DisabledPersonVetExemptionAmount,HomeownerExemptAmount,SeniorFreezeExemptAmount,LongtimeHomeownersExemptAmount,TaxYear,BillYear,LastPaymentDate1,LastPaymentDate2,PmtDue1,PmtDue2
2256,19103250470000,IL,CHICAGO,72014,0,0,0,1,22155.0,22155.0,...,0.0,0.0,0.0,0.0,2017.0,2017,00000000,00000000,2018-03-01,2018-08-01
2573774,19103250470000,IL,CHICAGO,72014,0,0,0,1,23811.0,23811.0,...,0.0,0.0,0.0,0.0,2018.0,2018,00000000,00000000,2019-03-01,2019-08-01
4860795,19103250470000,IL,CHICAGO,72014,0,0,0,1,24274.0,24274.0,...,0.0,0.0,0.0,0.0,2019.0,2019,00000000,00000000,2020-03-03,2020-08-03
6643148,19103250470000,IL,CHICAGO,72014,0,0,0,1,24690.0,24690.0,...,0.0,0.0,0.0,0.0,2020.0,2020,00000000,00000000,2021-03-02,2021-10-01
9263088,19103250470000,IL,CHICAGO,72014,0,0,0,1,27377.0,27377.0,...,0.0,0.0,0.0,0.0,2021.0,2021,00000000,00000000,2022-03-01,2023-01-03
10123118,19103250470000,IL,CHICAGO,72014,0,0,0,1,24553.0,24553.0,...,0.0,0.0,0.0,0.0,2022.0,2022,00000000,00000000,2023-04-03,NaN


In [13]:
print("Percentage of non-paid property not going to tax sold: {}%".format(df_concern1["PIN"].size*100/df_join["PIN"].size))
# df_concern1.to_csv('C:/Users/caoti/github/572CookCounty/Data/Cleaned_data/property_concern.csv',index=False,sep=',')

Percentage of non-paid property not going to tax sold: 3.2012228900892863%


In [14]:
# Converting date column to date time object
columns_to_convert1 = ['LastPaymentDate1', 'LastPaymentDate2']
columns_to_convert2 = ['PmtDue1','PmtDue2']
df_join[columns_to_convert2] = df_join[columns_to_convert2].apply(pd.to_datetime,format='%Y-%m-%d',  errors= 'coerce')

#Manually replace PmtDue2 day 08/03/2023 because InstDates.csv does not have value Inst2 for 2022
replacement_datetime = pd.to_datetime("2023-08-03")  # Replace with your desired datetime
df_join.loc[:,'PmtDue2'].fillna(replacement_datetime,inplace=True)

#For missing LastPaymentDate1 and LastPaymentDate2, set it to to PmdtDue1 date +1 or PmtDue2 date +1 appropriately.
df_join[columns_to_convert1] = df_join[columns_to_convert1].apply(pd.to_datetime,format='%m%d%Y',  errors= 'coerce')
df_join.loc[:,'LastPaymentDate1'].fillna(df_join.loc[:,'PmtDue1']+pd.DateOffset(days=1),inplace=True)
df_join.loc[:,'LastPaymentDate2'].fillna(df_join.loc[:,'PmtDue2']+pd.DateOffset(days=1),inplace=True)


In [15]:
df_join.iloc[-4:,-6:]

,TaxYear,BillYear,LastPaymentDate1,LastPaymentDate2,PmtDue1,PmtDue2
11191844,2022.0,2022,2023-03-30,2023-08-04,2023-04-03,2023-08-03
11191845,2022.0,2022,2023-03-30,2023-08-04,2023-04-03,2023-08-03
11191846,2022.0,2022,2023-03-30,2023-08-04,2023-04-03,2023-08-03
11191847,2022.0,2022,2023-03-29,2023-08-04,2023-04-03,2023-08-03


In [16]:
#Create flag whether the payment is late or not
df_join['LatePmt1'] = np.where(df_join['LastPaymentDate1'] > df_join['PmtDue1'], 1, 0)
df_join['LatePmt2'] = np.where(df_join['LastPaymentDate2'] > df_join['PmtDue2'], 1, 0)
df_join.iloc[-4:,-6:]

,LastPaymentDate1,LastPaymentDate2,PmtDue1,PmtDue2,LatePmt1,LatePmt2
11191844,2023-03-30,2023-08-04,2023-04-03,2023-08-03,0,1
11191845,2023-03-30,2023-08-04,2023-04-03,2023-08-03,0,1
11191846,2023-03-30,2023-08-04,2023-04-03,2023-08-03,0,1
11191847,2023-03-29,2023-08-04,2023-04-03,2023-08-03,0,1


In [17]:
#Set number of late payment features
df_join.loc[:,'DiffPmt1'] =  (df_join['LastPaymentDate1'] - df_join['PmtDue1']).dt.days
df_join.loc[:,'DiffPmt1'] = np.where(df_join['DiffPmt1'] > 0, df_join['DiffPmt1'], 0) #Change negative values to 0
df_join.iloc[-8:,-6:]

,LastPaymentDate2,PmtDue1,PmtDue2,LatePmt1,LatePmt2,DiffPmt1
11191840,2023-08-04,2023-04-03,2023-08-03,0,1,0
11191841,2023-08-04,2023-04-03,2023-08-03,0,1,0
11191842,2023-08-04,2023-04-03,2023-08-03,0,1,0
11191843,2023-08-04,2023-04-03,2023-08-03,0,1,0
11191844,2023-08-04,2023-04-03,2023-08-03,0,1,0
11191845,2023-08-04,2023-04-03,2023-08-03,0,1,0
11191846,2023-08-04,2023-04-03,2023-08-03,0,1,0
11191847,2023-08-04,2023-04-03,2023-08-03,0,1,0


In [18]:
df_join.loc[:,'PastTaxSaleStatus'] = np.where(df_join['PastTaxSaleStatus']=='Y',1,0) #1 for tax sale

In [19]:
#Columns to exclude from model training
exclude_cols = ['LastPaymentDate1','LastPaymentDate2','PmtDue1',
                'PmtDue2','TaxpayerMailingState','TaxpayerPropertyCity','CofENumber']
df_model = df_join.drop(exclude_cols,axis=1)
df_model.head(2)

,PIN,TaxCode,HomeownerExempt,SeniorExempt,SeniorFreezeExempt,BillType,AdjustedAmountDue1,TaxAmountDue1,CostAmountDue1,PastTaxSaleStatus,...,DisabledVetExemptionAmount,DisabledPersonVetExemptionAmount,HomeownerExemptAmount,SeniorFreezeExemptAmount,LongtimeHomeownersExemptAmount,TaxYear,BillYear,LatePmt1,LatePmt2,DiffPmt1
0,19084140790000,72001,1,0,0,1,133777.0,0.0,0.0,0,...,0.0,0.0,10000.0,0.0,0.0,2017.0,2017,1,0,15
1,19084140800000,72001,1,1,1,1,8952.0,0.0,0.0,0,...,0.0,0.0,10000.0,28515.0,0.0,2017.0,2017,0,1,0


In [20]:
#Initial Random Forest 
y=df_model['LatePmt2']
X= df_model.drop('LatePmt2',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

In [26]:
# Run with Task dataframe
df_dask = dd.from_pandas(df_model, npartitions=90)  # Specify the number of partitions 

X = df_dask.drop('LatePmt2', axis=1)  # Replace 'target_column' with the name of your target column
y = df_dask['LatePmt2']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train.compute(), y_train.compute())  # Compute the Dask DataFrames to trigger the computation

RandomForestClassifier()

In [27]:
import pickle
path_to_save = "C:/Users/caoti/github/572CookCounty/Data/Model/RandomForest_100Esti.pkl"
# Save the model info as binary format
with open(path_to_save, 'wb') as file:
    pickle.dump(clf, file)


In [28]:
# Make predictions
y_pred = clf.predict(X.compute())

In [33]:
# Extract feature importances
importances = clf.feature_importances_

# Create a DataFrame with feature names and importances
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances})

# Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False)
# Reset the index of the DataFrame
feature_importance_df = feature_importance_df.reset_index(drop=True)
# Print the model information and feature importances
print("Model Information:")
print(clf)
print("\nFeature Importances:")
print(feature_importance_df)

Model Information:
RandomForestClassifier()

Feature Importances:
                             Feature    Importance
0                            TaxRate  1.962896e-01
1                 AdjustedAmountDue1  1.534090e-01
2                            TaxYear  1.214815e-01
3                           DiffPmt1  1.199974e-01
4                           BillYear  9.681514e-02
5                           LatePmt1  9.259157e-02
6                                PIN  8.501081e-02
7                    HomeownerExempt  2.546128e-02
8              HomeownerExemptAmount  2.545754e-02
9                      TaxAmountDue1  2.009486e-02
10                           TaxCode  1.751308e-02
11                          BillType  1.634307e-02
12                 PastTaxSaleStatus  7.595043e-03
13          SeniorFreezeExemptAmount  6.309711e-03
14                      ExemptStatus  5.426424e-03
15                    CostAmountDue1  5.015213e-03
16        DisabledVetExemptionAmount  1.516609e-03
17              

In [37]:
# Get predicted probabilities for the positive class
y_pred_probs = clf.predict_proba(X_test)[:, 1]
y_pred_probs

array([0.05, 0.  , 0.25, ..., 1.  , 1.  , 1.  ])

In [38]:
#nums is the probability threshold
nums = [0.2, 0.25, 0.3, 0.35, 0.4, 0.5]

# Initialize an empty list to store the evaluation metrics
metrics_list = []

# Iterate through each threshold
for threshold in nums:
    # Convert probabilities to binary predictions based on the threshold
    y_pred = [1 if prob >= threshold else 0 for prob in y_pred_probs]

    # Calculate the evaluation metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    # Append the metrics to the list
    metrics_list.append([threshold, accuracy, f1, precision, recall])

# Create a DataFrame from the metrics list
metrics_df = pd.DataFrame(metrics_list, columns=['Threshold', 'Accuracy', 'F1 Score', 'Precision', 'Recall'])

# Print the metrics DataFrame
print(metrics_df)

   Threshold  Accuracy  F1 Score  Precision    Recall
0       0.20  0.875383  0.834323   0.781624  0.894641
1       0.25  0.892479  0.852353   0.822127  0.884887
2       0.30  0.904319  0.865296   0.854647  0.876213
3       0.35  0.912963  0.874970   0.881712  0.868331
4       0.40  0.918792  0.881452   0.903120  0.860800
5       0.50  0.926324  0.889522   0.938154  0.845683


In [71]:
df_model[df_model['DiffPmt1']==0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8860876 entries, 1 to 11191847
Data columns (total 29 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   PIN                               int64  
 1   TaxCode                           int32  
 2   HomeownerExempt                   int32  
 3   SeniorExempt                      int32  
 4   SeniorFreezeExempt                int32  
 5   BillType                          int32  
 6   AdjustedAmountDue1                float64
 7   TaxAmountDue1                     float64
 8   CostAmountDue1                    float64
 9   PastTaxSaleStatus                 int32  
 10  TaxRate                           float64
 11  CondemnationStatus                int32  
 12  MunicipalAcquisitionStatus        int32  
 13  AcquisitionStatus                 int32  
 14  ExemptStatus                      int32  
 15  BankruptStatus                    int32  
 16  RefundStatus                      i

In [91]:
Original = df_join["PIN"].size
tem = (df_join["DiffPmt1"]==0).sum()
print("Original dataset: {}. Total DiffPmt1: {}. Percentage of late: {}".format(total, tem, 100-(tem*100/total)))

Original dataset: 11191848. Total DiffPmt1: 8860876. Percentage of late: 20.827409378683484
